### Document Loaders

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./bella_vista.txt")
docs = loader.load()

In [3]:
print(docs)
print(len(docs))

[Document(page_content="Q: What are the hours of operation for Bella Vista?\nA: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome guests from 12 p.m. to 10 p.m.\n\nQ: What type of cuisine does Bella Vista serve?\nA: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.\n\nQ: Do you offer vegetarian or vegan options at Bella Vista?\nA: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and vegan dishes. Our chefs are also happy to customize dishes based on dietary needs.\n\nQ: Is Bella Vista family-friendly?\nA: Yes, Bella Vista is a family-friendly establishment. We have a dedicated kids' menu and offer high chairs and booster seats for our younger guests.\n\nQ: Can I book private events at Bella Vista?\nA: Certainly! Bella Vista has a private dining area perfect for events, parties, or co

In [4]:
from langchain.schema import Document

example_doc = Document(page_content="test", metadata={"important_info": "hi there"})
example_doc

Document(page_content='test', metadata={'important_info': 'hi there'})

Texts are not loaded 1:1 into the database, but in pieces, so called "chunks". You can define the chunk size and the overlap between the chunks.

To create multiple documents (chunks), you can use a text splitter.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

In [6]:
for doc in documents:
    print(doc)

len(documents)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='Q: What type of cuisine does Bella Vista serve?' metadata={'source': './bella_vista.txt'}
page_content='A: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We' metadata={'source': './bella_vista.txt'}
page_content='cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.' metadata={'source': './bella_vista.txt'}
page_content='Q: Do you offer vegetarian or vegan options at Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='A: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and veg

22

### Embeddings

Texts are not stored as text in the database, but as vector representations. 
Embeddings are a type of word representation that represents the semantic meaning of words in a vector space.

In [7]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [8]:
embedding1 = embeddings.embed_query(text="The solar system consists of the Sun and the objects that orbit it")
print(embedding1)
print(len(embedding1))

[0.02494740092479872, 0.021047435522573793, -0.010201815429324048, -0.02243408914275127, -0.040312029079795, 0.022372184131741055, -0.005441381084266655, -0.016256047275027444, -0.0011351685874847078, -0.009421822535143575, 0.008951350039401344, 0.005370191252927474, -0.014436064165713865, 0.00196081607803925, -0.00595518615639347, -0.015451293307765453, 0.01119228256697155, -0.004178534692141908, 0.0063885156455295725, -0.0037544909546573376, 0.0023337891127629606, -0.0069704152984450575, -0.011433709315943686, -0.00016791520323368702, 0.01724651627532008, -0.004147582652298084, 0.011650374060511738, -0.03191781389493736, 0.0228178946230792, -0.009378489027436425, 0.020267441697070756, -0.010870381166331267, -0.01992077736070382, -0.015537959391857186, 0.015525578389655143, 0.004079488071509415, 0.007149937502068263, 0.009985150684755997, -0.03206638592136188, 0.004153773153399105, 0.011464661821448794, 0.03362637357236795, -0.004707816017021277, -0.005317572459230075, -0.012789411361

In [9]:
embedding2 = embeddings.embed_query(text="The solar system consists of the Sun and the objects that orbit it")
embedding3 = embeddings.embed_query(text="Planets, asteroids, and comets are part of our solar system.")
embedding4 = embeddings.embed_query(text="I love baking chocolate chip cookies on weekends.")

In [10]:
import numpy as np

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    return dot_product / (norm_a * norm_b)

In [11]:
sim_1_2 = cosine_similarity(embedding1, embedding2)
sim_1_3 = cosine_similarity(embedding1, embedding3)
sim_3_4 = cosine_similarity(embedding3, embedding4)

print(sim_1_2, sim_1_3, sim_3_4)


0.9999720839266372 0.9010542982624454 0.6975982883904684


### Loading Vectors into VectorDB (FAISS)

As created by OpenAIEmbeddings vectors and documents can now be stored in the database. This DB can be stored as .pkl file

In [12]:
from langchain.vectorstores.faiss import FAISS
import pickle

vectorstore = FAISS.from_documents(documents, embeddings)

with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

### Loading the database

Before using the database, it must of course be loaded again.

In [13]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)


In [14]:
retriever = vectorstore.as_retriever()

In [15]:
docs = retriever.get_relevant_documents(query="When are the opening hours??")
for doc in docs:
    print(doc)


page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends' metadata={'source': './bella_vista.txt'}
page_content='during weekends and holidays, to ensure a seamless dining experience.' metadata={'source': './bella_vista.txt'}


In [16]:
docs = retriever.get_relevant_documents(query="When are the opening hours?", filter={'source': './bella_vista.txt'}, k=3)
for doc in docs:
    print(doc) # does not work!

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}
page_content='Sundays, we welcome guests from 12 p.m. to 10 p.m.' metadata={'source': './bella_vista.txt'}
page_content='A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends' metadata={'source': './bella_vista.txt'}
page_content='during weekends and holidays, to ensure a seamless dining experience.' metadata={'source': './bella_vista.txt'}


In [17]:
retriever = vectorstore.as_retriever(search_kwargs={"filter": {'source': './bella_vista.txt'}, "k":1})
docs = retriever.get_relevant_documents(query="When are the opening hours??")
for doc in docs:
    print(doc)

page_content='Q: What are the hours of operation for Bella Vista?' metadata={'source': './bella_vista.txt'}


### Now we have to pass the documents to an LLM.

We create a prompt with a question and context. Context is the output from the retriever (Document Store).
LangChain provides chains out-of-the-box to do that, the RetrievalChains

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant for our restaurant.

{context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

llm = ChatOpenAI()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

result = qa.run(query="When are the opening hours on sunday??")
print(result)

The opening hours on Sunday are from 12 p.m. to 10 p.m.
